In [ ]:
class CFG:
    wandb=True
    competition='NBME'
    _wandb_kernel='nakama'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-base"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=12
    fc_dropout=0.2
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [2]:
import pandas as pd

data = pd.read_csv('train_data_dataframe.csv')

display(data)



,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False
2,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_erOlnfgWx2,IMD는 스위스의 비영리 민간기구로 국가경쟁력의 순위를 1996년이래 연례적으로 발...,QUES_WwhEDEQ4JT,국가경쟁력의 순위를 1996년 이전 연례적으로 발표하는 스위스의 비영리 민간기구는 뭐야,NaN,NaN,True
3,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_pP8k4n8yg4,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,QUES_GnQIVxXt70,기술무역액 등과 같이 가용할 수 있는 통계 수치에 전적으로 의존하는 평가가 뭐야,종합과학기술력 평가,124.0,False
4,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_7b3M3mKZbb,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...",QUES_RcrMURMuhd,창의력을 저해하는 인사제도 요인은 뭐야,연공서열제,120.0,False
...,...,...,...,...,...,...,...,...,...
37324,CNTS_2129305745,국역 금대전책,PARS_ma0DB2fJSk,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,QUES_XM4RUoboD7,역상을 알지 못함을 분개하면서 이에 대한 견해를 토로하도록 하는 책문을 정조가 내놓...,천체론,1.0,False
37325,CNTS_2129305745,국역 금대전책,PARS_wGlvzy6QkR,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,QUES_YsPrUyg1xE,사대부들이 정학을 기피하면서 기교에 기반한 글쓰기를 하자 문체반정을 추진한 왕은 누구야,정조,244.0,False
37326,CNTS_7969874543,과학기술 ODA 사업의 사례2 농업기술분야-몽골 강원도 농업타운 조성 및 운영 성...,PARS_wXIJMVVRcQ,또한 강원도농업타운의 조성 및 운영의 궁극적인 목적이 몽골 현지에서의 시범 재배 및...,QUES_Lz55ir98nY,강원도농업기술원의 육류 전문가를 2004년에 몽골에 몇 개월간 파견하였냐,NaN,NaN,True
37327,CNTS_7969874543,과학기술 ODA 사업의 사례2 농업기술분야-몽골 강원도 농업타운 조성 및 운영 성...,PARS_Tk1KaMM2tz,강원도의 몽골 농업기술교류사업 중 성공 모델로 각광받고 있는 강원도농업타운 조성의 ...,QUES_dklx3PbuRp,매년 몽골의 작물 재배상황이 일정해서 무엇의 확립이 어려웠지,NaN,NaN,True


array(['기술경쟁력', '평균 32위', nan, ..., '달', '250리', '천체론'], dtype=object)

In [4]:
data['answer_text'].unique()

array(['기술경쟁력', '평균 32위', nan, ..., '달', '250리', '천체론'], dtype=object)